In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bile_Duct_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Bile_Duct_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Bile_Duct_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Bile_Duct_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Bile_Duct_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# List all subdirectories in tcga_root_dir
subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA directories: {subdirs}")

# Look for the directory matching Bile Duct Cancer
bile_duct_dir = None
for dir_name in subdirs:
    if 'Bile_Duct_Cancer' in dir_name or 'CHOL' in dir_name:
        bile_duct_dir = dir_name
        break

if bile_duct_dir:
    print(f"Found matching directory for {trait}: {bile_duct_dir}")
    cohort_dir = os.path.join(tcga_root_dir, bile_duct_dir)
    
    # Get paths to clinical and genetic files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Load clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    print(f"\nSelected directory: {bile_duct_dir}")
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check if gene expression and trait data are available
    is_gene_available = not genetic_df.empty
    is_trait_available = not clinical_df.empty
    
    # Only validate, don't finalize
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
else:
    print(f"No suitable directory found for {trait}.")
    
    # Mark this cohort as not usable for Bile Duct Cancer research
    validate_and_save_cohort_info(
        is_final=False, 
        cohort="TCGA", 
        info_path=json_path, 
        is_gene_available=False, 
        is_trait_available=False,
    )


Available TCGA directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Load clinical data to preview these columns
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Bile_Duct_Cancer_(CHOL)")
clinical_file, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file, sep='\t', index_col=0)

# Preview age columns
if candidate_age_cols:
    age_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_age_cols if col in clinical_df.columns}
    print("Age columns preview:")
    print(age_preview)

# Preview gender columns
if candidate_gender_cols:
    gender_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_gender_cols if col in clinical_df.columns}
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [72, 50, 70, 72, 60], 'days_to_birth': [-26349, -18303, -25819, -26493, -21943]}
Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examining the age columns
age_columns = {'age_at_initial_pathologic_diagnosis': [72, 50, 70, 72, 60], 
               'days_to_birth': [-26349, -18303, -25819, -26493, -21943]}

# Examining the gender columns
gender_columns = {'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}

# Select the best columns for age and gender
# For age, we prefer 'age_at_initial_pathologic_diagnosis' as it provides direct age values
# 'days_to_birth' provides negative values representing days from birth (would need conversion)
age_col = 'age_at_initial_pathologic_diagnosis' if 'age_at_initial_pathologic_diagnosis' in age_columns else None

# For gender, 'gender' is the only available column and appears to have valid values
gender_col = 'gender' if 'gender' in gender_columns else None

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step: Feature Engineering and Validation

# 1. Extract and standardize clinical features
print("\nExtracting clinical features...")
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

print(f"Clinical features shape: {clinical_features.shape}")
print(f"Preview of clinical features:\n{clinical_features.head()}")

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Normalize gene symbols in the gene expression data
print("\nNormalizing gene expression data...")
genetic_df_normalized = normalize_gene_symbols_in_index(genetic_df)
print(f"Original gene expression data shape: {genetic_df.shape}")
print(f"Normalized gene expression data shape: {genetic_df_normalized.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 3. Link the clinical and genetic data on sample IDs
print("\nLinking clinical and genetic data...")
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_for_linking = genetic_df_normalized.T

# Ensure sample IDs in clinical features match those in genetic data
common_samples = clinical_features.index.intersection(genetic_df_for_linking.index)
print(f"Number of common samples: {len(common_samples)}")

# Filter both dataframes to keep only common samples
clinical_features_common = clinical_features.loc[common_samples]
genetic_df_common = genetic_df_for_linking.loc[common_samples]

# Combine clinical and genetic data
linked_data = pd.concat([clinical_features_common, genetic_df_common], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values systematically
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data_clean.shape}")

# 5. Determine if trait or demographic features are biased
print("\nChecking for biased features...")
is_trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Validate data quality and save cohort information
print("\nValidating final data quality...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=genetic_df_normalized.shape[0] > 0,
    is_trait_available=clinical_features.shape[0] > 0,
    is_biased=is_trait_biased,
    df=linked_data_clean,
    note="Pancreatic cancer dataset used as proxy for Type 2 Diabetes due to pancreatic involvement in diabetes."
)

# 7. Save the linked data if usable
if is_usable:
    print(f"\nData is usable. Saving linked data to {out_data_file}...")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("\nData is not usable. Linked data will not be saved.")


Extracting clinical features...
Clinical features shape: (45, 3)
Preview of clinical features:
                 Bile_Duct_Cancer  Age  Gender
sampleID                                      
TCGA-3X-AAV9-01                 1   72       1
TCGA-3X-AAVA-01                 1   50       0
TCGA-3X-AAVB-01                 1   70       0
TCGA-3X-AAVC-01                 1   72       0
TCGA-3X-AAVE-01                 1   60       1
Clinical data saved to ../../output/preprocess/Bile_Duct_Cancer/clinical_data/TCGA.csv

Normalizing gene expression data...


Original gene expression data shape: (20530, 45)
Normalized gene expression data shape: (19848, 45)


Normalized gene expression data saved to ../../output/preprocess/Bile_Duct_Cancer/gene_data/TCGA.csv

Linking clinical and genetic data...
Number of common samples: 45
Linked data shape: (45, 19851)

Handling missing values...


Data shape after handling missing values: (45, 19851)

Checking for biased features...
For the feature 'Bile_Duct_Cancer', the least common label is '0' with 9 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Bile_Duct_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 68.0
  75%: 73.0
Min: 29
Max: 82
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 22 occurrences. This represents 48.89% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Validating final data quality...

Data is usable. Saving linked data to ../../output/preprocess/Bile_Duct_Cancer/TCGA.csv...


Linked data saved to ../../output/preprocess/Bile_Duct_Cancer/TCGA.csv
